# <div style="padding:20px;color:white;margin:0;font-size:100%;text-align:left;display:fill;border-radius:5px;background-color:#6A79BA;overflow:hidden">Data Filter</div>

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', 1000)
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')
# Stop words
stop_words = set(stopwords.words('english'))

df0=pd.read_csv('result/data.csv')

[nltk_data] Downloading package stopwords to /Users/mmcao/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/mmcao/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Delete samples with too short titles

In [2]:
df = df0.copy()
# filter out short news with too short title
word_num_thres = 6
df = df[df['title'].apply(lambda x: len(x.split()) >= word_num_thres)]

# delete stopwords
# df['title'] = df['title'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word.lower() not in stop_words]))

# Calculate the deleted results by keywords
df_deleted_by_too_short_titles = df0[~df0.index.isin(df.index)]

In [3]:
df_deleted_by_too_short_titles

,date,timestamp,sym,title
13,1/2/2024,56:48.0,GBP,U.S. FX Markets Open-January 2


## Method 1： Keywords-Based

In [4]:
keywords = ['GBP/USD', 'forex', 'stocks', 'economy', 'GBP', 'fed', 'USD', 'british', 'sterling', 'euro', 'UST yields', 'dollar', 
            'inflation', 'U.S. yields', 'EUR', 'pound', 'exchange rate']  # 定义关键词列表

# filtered_df_kw = df[df['title'].apply(lambda x: (any(keyword.lower() in x.lower() for keyword in keywords) or any(re.search(r'\b' + re.escape(keyword.lower()) + r'\b', x.lower()) for keyword in keywords)))]
filtered_df_kw = df[df['title'].str.contains('|'.join(keywords), case=False)] 
# Calculate the deleted results by keywords
df_deleted_by_kw = df[~df.index.isin(filtered_df_kw.index)]

# Output deleted and filtered DataFrame
filtered_df_kw.to_csv('result/filtered_df_kw.csv', index=False)
filtered_df_kw.to_excel('result/filtered_df_kw.xlsx', index=False)
df_deleted_by_kw.to_csv('result/df_deleted_by_kw.csv', index=False)
df_deleted_by_kw.to_excel('result/df_deleted_by_kw.xlsx', index=False)

In [5]:
df_deleted_by_kw

,date,timestamp,sym,title
0,1/1/2024,33:57.0,GBP,BUZZ-Asia Day Ahead-Focus on Xi comments and busy data schedule
12,1/2/2024,56:27.0,GBP,BUZZ-U.S. FX Markets Open- January 2
15,1/2/2024,00:00.0,GBP,REG-VinaCapital Vietnam- Daily Net Asset Value
20,1/2/2024,14:32.0,GBP,BUZZ-Larger option expiries for Tuesday's New York cut


# Method 2: TF-IDF feature extraction

In [6]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['title'])
feature_names = tfidf.get_feature_names_out()

# Calculate the average TF-IDF value for each sample
average_tfidf_values = tfidf_matrix.mean(axis=1).flatten()
average_tfidf_values = average_tfidf_values.A1.tolist()

# Calculate the number of samples to delete based on the delete_ratio
delete_ratio = 0.1  # 10% delete ratio
num_samples_to_delete = int(delete_ratio * len(df))

# Sort the average TF-IDF values in descending order
sorted_tfidf_values = np.sort(average_tfidf_values)

# Calculate the new threshold based on the delete_ratio
new_threshold = sorted_tfidf_values[num_samples_to_delete]

# Filter data based on the threshold
filtered_indices = [idx for idx, val in enumerate(average_tfidf_values) if val >= new_threshold]
filtered_df_tfidf = df.iloc[filtered_indices]

# Calculate the deleted results by TF-IDF
df_deleted_by_tfidf = df[~df.index.isin(filtered_df_tfidf.index)]

# Output filtered DataFrame
filtered_df_tfidf.to_csv('result/filtered_df_tfidf.csv', index=False)
filtered_df_tfidf.to_excel('result/filtered_df_tfidf.xlsx', index=False)
df_deleted_by_tfidf.to_csv('result/df_deleted_by_tfidf.csv', index=False)
df_deleted_by_tfidf.to_excel('result/df_deleted_by_tfidf.xlsx', index=False)

In [7]:
df_deleted_by_tfidf

,date,timestamp,sym,title
10,1/2/2024,09:25.0,GBP,BUZZ-COMMENT-Will investors vote for sterling this year?
12,1/2/2024,56:27.0,GBP,BUZZ-U.S. FX Markets Open- January 2


## Method 3: LDA topic modeling

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = stopwords.words('english')

# Assuming df is your DataFrame containing the 'title' column
titles = df['title'].tolist()

# Preprocess the text
vectorizer = CountVectorizer(stop_words=stop_words)
X = vectorizer.fit_transform(titles)

# Set the number of topics
n_topics = 10

# Train the LDA model
lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda.fit(X)

# Get the vocabulary
vocab = vectorizer.get_feature_names_out()

# Print the top 5 keywords for each topic
for i, topic in enumerate(lda.components_):
    print(f"Topic {i}:")
    print(" ".join([vocab[i] for i in topic.argsort()[:-6:-1]]))

[nltk_data] Downloading package stopwords to /Users/mmcao/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Topic 0:
usd dollar buzz forex starts2024
Topic 1:
buzz forex starts2024 dollar usd
Topic 2:
buzz usd gbp 27 ust
Topic 3:
buzz eur gbp clouding versus
Topic 4:
buzz focus data yields sterling
Topic 5:
new option larger expiries tuesday
Topic 6:
dollar year buzz new york
Topic 7:
forex starts2024 bid yen gains
Topic 8:
buzz markets open january fx
Topic 9:
year sterling vote buzz comment


In [9]:
# Manually select the topics to exclude (assuming we want to exclude topics 0 and 3)
exclude_topics = [5, 7]

# Get the topic distribution for each title
topic_distribution = lda.transform(X)

# Retain samples that do not belong to the excluded topics
mask = ~np.isin(topic_distribution.argmax(axis=1), exclude_topics)
filtered_df_lda = df[mask]
df_deleted_by_lda = df[~mask]

# Calculate the deleted results by LDA
df_deleted_by_lda = df[~df.index.isin(filtered_df_lda.index)]

filtered_df_lda.to_csv('result/filtered_df_lda.csv', index=False)
filtered_df_lda.to_excel('result/filtered_df_lda.xlsx', index=False)
df_deleted_by_lda.to_csv('result/df_deleted_by_lda.csv', index=False)
df_deleted_by_lda.to_excel('result/df_deleted_by_lda.xlsx', index=False)
columns=['date', 'timestamp', 'sym', 'title']
filtered_df_lda = filtered_df_lda[columns]

In [10]:
df_deleted_by_lda

,date,timestamp,sym,title
4,1/2/2024,00:00.0,GBP,ANALYSIS-Sterling runs into economic and election hurdles after stellar year
6,1/2/2024,43:34.0,GBP,BUZZ-GBP/USD tentatively bid but signals are mixed
9,1/2/2024,06:38.0,GBP,"FOREX-Dollar starts2024 with gains on yen and euro, bitcoin above $45,000"
15,1/2/2024,00:00.0,GBP,REG-VinaCapital Vietnam- Daily Net Asset Value
20,1/2/2024,14:32.0,GBP,BUZZ-Larger option expiries for Tuesday's New York cut


In [11]:
# Calculate the overall union
union_df = pd.concat([filtered_df_kw, filtered_df_tfidf, filtered_df_lda]).drop_duplicates()

# Calculate the deleted results
df_three_methods_deleted = df0[~df0.index.isin(union_df.index)]

# Calculate the union and intersection between different filtered DataFrames
union_kw_tfidf = pd.concat([filtered_df_kw, filtered_df_tfidf]).drop_duplicates()
union_kw_lda = pd.concat([filtered_df_kw, filtered_df_lda]).drop_duplicates()
union_tfidf_lda = pd.concat([filtered_df_tfidf, filtered_df_lda]).drop_duplicates()

intersection_kw_tfidf = pd.merge(filtered_df_kw, filtered_df_tfidf, how='inner')
intersection_kw_lda = pd.merge(filtered_df_kw, filtered_df_lda, how='inner')
intersection_tfidf_lda = pd.merge(filtered_df_tfidf, filtered_df_lda, how='inner')

# Store the overall union as CSV and Excel files
union_df.to_csv('result/union_df.csv', index=False)
union_df.to_excel('result/union_df.xlsx', index=False)

# Store as CSV and Excel files
df_three_methods_deleted.to_csv('result/df_three_methods_deleted.csv', index=False)
df_three_methods_deleted.to_excel('result/df_three_methods_deleted.xlsx', index=False)

union_kw_tfidf.to_csv('result/union_kw_tfidf.csv', index=False)
union_kw_tfidf.to_excel('result/union_kw_tfidf.xlsx', index=False)
union_kw_lda.to_csv('result/union_kw_lda.csv', index=False)
union_kw_lda.to_excel('result/union_kw_lda.xlsx', index=False)
union_tfidf_lda.to_csv('result/union_tfidf_lda.csv', index=False)
union_tfidf_lda.to_excel('result/union_tfidf_lda.xlsx', index=False)
intersection_kw_tfidf.to_csv('result/intersection_kw_tfidf.csv', index=False)
intersection_kw_tfidf.to_excel('result/intersection_kw_tfidf.xlsx', index=False)
intersection_kw_lda.to_csv('result/intersection_kw_lda.csv', index=False)
intersection_kw_lda.to_excel('result/intersection_kw_lda.xlsx', index=False)
intersection_tfidf_lda.to_csv('result/intersection_tfidf_lda.csv', index=False)
intersection_tfidf_lda.to_excel('result/intersection_tfidf_lda.xlsx', index=False)

# Final summary
print("\nSummary:")
print("Orignal DataFrame Rows:", len(df0))
print("Orignal DataFrame Rows after min word number limit:", len(df))
print("Union DataFrame Rows:", len(union_df))
print("Three Methods Deleted DataFrame Rows:", len(df_three_methods_deleted))
print("Union of Keyword and TF-IDF DataFrames Rows:", len(union_kw_tfidf))
print("Union of Keyword and LDA DataFrames Rows:", len(union_kw_lda))
print("Union of TF-IDF and LDA DataFrames Rows:", len(union_tfidf_lda))
print("Intersection between Keyword and TF-IDF DataFrames Rows:", len(intersection_kw_tfidf))
print("Intersection between Keyword and LDA DataFrames Rows:", len(intersection_kw_lda))
print("Intersection between TF-IDF and LDA DataFrames Rows:", len(intersection_tfidf_lda))


Summary:
Orignal DataFrame Rows: 22
Orignal DataFrame Rows after min word number limit: 21
Union DataFrame Rows: 21
Three Methods Deleted DataFrame Rows: 1
Union of Keyword and TF-IDF DataFrames Rows: 20
Union of Keyword and LDA DataFrames Rows: 19
Union of TF-IDF and LDA DataFrames Rows: 21
Intersection between Keyword and TF-IDF DataFrames Rows: 16
Intersection between Keyword and LDA DataFrames Rows: 14
Intersection between TF-IDF and LDA DataFrames Rows: 14


In [12]:
df_three_methods_deleted

,date,timestamp,sym,title
13,1/2/2024,56:48.0,GBP,U.S. FX Markets Open-January 2
